In [1]:
import pandas as pd
import numpy as np
import subprocess
from common import data2libffm  #需将data hash成ffm输入文件格式 1:2:3
import gc

In [7]:
train = pd.read_pickle("../data/merge_data/final_train.pickle")

In [8]:
train.rename(columns={"is_trade_x":"label"},inplace=True) 

In [19]:
set(train.columns)

{'7_click_cnt',
 '8_click_cnt',
 'adClick',
 'context_id',
 'context_page_id',
 'context_timestamp',
 'dayClick_currence',
 'dayClick_max',
 'dayClick_mean',
 'dayClick_median',
 'dayClick_min',
 'dist_max',
 'dist_mean',
 'dist_median',
 'dist_min',
 'dist_std',
 'first_2_last_dist',
 'hour',
 'instance_id',
 'item_brand_id',
 'item_brand_id_sum_count',
 'item_category_list',
 'item_category_list_sum_count',
 'item_city_id',
 'item_city_id_sum_count',
 'item_collected_level',
 'item_id',
 'item_id_sum_count',
 'item_price_level',
 'item_property_list',
 'item_property_list_sum_count',
 'item_pv_level',
 'item_sales_level',
 'label',
 'last_click_1_2_dist',
 'last_click_1_3_dist',
 'last_click_2_3_dist',
 'now_2_first_dist',
 'now_2_last_dist',
 'predict_category_property',
 'shop_id',
 'shop_id_sum_count',
 'shop_review_num_level',
 'shop_review_positive_rate',
 'shop_score_delivery',
 'shop_score_description',
 'shop_score_service',
 'shop_star_level',
 'user_age_level',
 'user_gende

In [1]:
del train

In [20]:
drop = ['context_page_id',
 'context_timestamp',
 'hour',
 'item_brand_id',
 'item_category_list',
 'item_city_id',
 'item_id',
 'item_property_list',
 'shop_id',
 'user_id','dayClick_median',]
train = train.drop(drop,axis=1)

In [21]:
train.shape

(3011618, 45)

In [9]:
test = pd.read_pickle("../data/merge_data/final_test.pickle")

In [11]:
test_index = test.set_index("instance_id")

In [12]:
test.shape

(1209768, 46)

In [13]:
test["label"] = -1

In [14]:
test = test.drop(["day"],axis=1)

In [23]:
test = test.drop(["dayClick_median"],axis=1)

In [24]:
test.shape

(1209768, 45)

In [13]:
test = test.drop(drop_ts,axis=1)

In [26]:
data2libffm(test,'online_test.ffm') #对data hash化
del test
gc.collect()

0it [00:00, ?it/s]    60    0m
999566it [03:20, 4996.59it/s]   260    1m
1209768it [04:01, 5005.44it/s]


finish convert data to libffm


23

In [25]:
data2libffm(train,"online_train.ffm")
del train
gc.collect()

0it [00:00, ?it/s]  2827    0m
999967it [13:54, 1198.40it/s]  3661    1m
1999969it [31:17, 1065.38it/s]  4704    2m
2999889it [52:30, 952.15it/s]   5978    3m
3011618it [52:44, 951.76it/s]


finish convert data to libffm


3138

In [57]:
train_path = '../data/Field-aware/online_train.ffm'
val_path = "../data/Field-aware/online_val.ffm"
test_path = '../data/Field-aware/online_test.ffm'
model_path = '../data/model/ffm.model'
result_path = '../data/model/online_pred.csv'

#./ffm-train -r 0.05 -t 23 -s 20 -l 0.0000005 train_path model_path
#./ffm-predict test_path model_path online_pred.csv
subprocess.call('ffm-train -r 0.05 -t 12 -s 20 -l 0.0000005 {0} {1}'.format(train_path,model_path)) #epoch
subprocess.call('ffm-predict {0} {1} {2}'.format(test_path,model_path,result_path))

0

In [2]:
ans = pd.read_csv('../data/model/ffm_pred.csv',names=['prob'])
ans["prob"].mean()

In [ ]:
#18:0.047 16:0.050 15:0.040 14:0.03498 13:0.03684 12:0.03677

In [59]:
res = pd.DataFrame(test_index.index)
res["predicted_score"] = ans["prob"]

In [60]:
res.to_csv("../data/result/ffm_preds.txt",sep=" ",index=False,line_terminator="\r")

In [3]:
res.head(10)